In [6]:
import numpy as np
import pandas as pd
import cv2
from utilities import save_video_from_images

In [3]:
dataset_location = '../dataset/' # put the root or relative path to your dataset here

In [5]:
scene_df1 = pd.read_hdf(dataset_location + 'scene.hdf5')
scene_df2 = pd.read_hdf(dataset_location + 'scene2.hdf5')
scene_df3 = pd.read_hdf(dataset_location + 'scene3.hdf5')
car_scene_df1 = pd.read_hdf(dataset_location + 'car_scene.hdf5')
car_scene_df2 = pd.read_hdf(dataset_location + 'car_scene2.hdf5')
car_scene_df3 = pd.read_hdf(dataset_location + 'car_scene3.hdf5')

In [11]:
def filter_frequency_domain(frames, cutoff_frequency=30):
    denoised_frames = []
    for frame in frames:
        dft = cv2.dft(np.float32(frame), flags=cv2.DFT_COMPLEX_OUTPUT)
        dft_shift = np.fft.fftshift(dft)
        
        rows, cols = frame.shape
        crow, ccol = rows // 2, cols // 2
        
        mask = np.zeros((rows, cols, 2), np.uint8)
        mask[crow - cutoff_frequency:crow + cutoff_frequency, ccol - cutoff_frequency:ccol + cutoff_frequency] = 1
        
        fshift = dft_shift * mask
        f_ishift = np.fft.ifftshift(fshift)
        img_back = cv2.idft(f_ishift)
        img_back = cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])
        
        denoised_frames.append(img_back)
    
    return denoised_frames

In [12]:
all_scenes = [scene_df1, scene_df2, scene_df3, car_scene_df1, car_scene_df2, car_scene_df3]

for i, scene in enumerate(all_scenes):
    frames = scene['image'].values
    frames = np.array([np.array(frame) for frame in frames])
    denoised = filter_frequency_domain(frames)
    location = "frequency_filtering/"
    name = f"scene{i+1}.avi" if i < 3 else f"car_scene{i-2}.avi"
    save_video_from_images(denoised, location + name)